# 词袋模型（Bag of Words Model）

词袋模型看起来好像一个口袋把所有词都装进去，但却不完全如此。在自然语言处理和信息检索中作为一种简单假设，词袋模型把文本（段落或者文档）被看作是无序的词汇集合，忽略语法甚至是单词的顺序，把每一个单词都进行统计，同时计算每个单词出现的次数，常常被用在文本分类中，如贝叶斯算法、LDA 和 LSA 等。

## 动手实战词袋模型

In [1]:
import jieba

punctuation = ["，","。", "：", "；", "？"]
content = ["机器学习带动人工智能飞速的发展。", 
           "深度学习带动人工智能飞速的发展。", 
           "机器学习和深度学习带动人工智能飞速的发展。"]

segs_l = [jieba.lcut(con) for con in content]
segs_l

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.836 seconds.
Prefix dict has been built succesfully.


[['机器', '学习', '带动', '人工智能', '飞速', '的', '发展', '。'],
 ['深度', '学习', '带动', '人工智能', '飞速', '的', '发展', '。'],
 ['机器', '学习', '和', '深度', '学习', '带动', '人工智能', '飞速', '的', '发展', '。']]

In [7]:
tokenized = []
for sentence in segs_l:
    words = []
    for word in sentence:
        if word not in punctuation:
            words.append(word)
    tokenized.append(words)
tokenized

[['机器', '学习', '带动', '人工智能', '飞速', '的', '发展'],
 ['深度', '学习', '带动', '人工智能', '飞速', '的', '发展'],
 ['机器', '学习', '和', '深度', '学习', '带动', '人工智能', '飞速', '的', '发展']]

In [5]:
bag_of_words = [x for item in segs_l for x in item if x not in punctuation]
" ".join(bag_of_words)

'机器 学习 带动 人工智能 飞速 的 发展 深度 学习 带动 人工智能 飞速 的 发展 机器 学习 和 深度 学习 带动 人工智能 飞速 的 发展'

In [6]:
bag_of_words = list(set(bag_of_words))
bag_of_words

['深度', '飞速', '发展', '和', '人工智能', '的', '带动', '学习', '机器']

In [8]:
bag_of_words2vec = []
for sentence in tokenized:
    tokens = [1 if token in sentence else 0 for token in bag_of_words]
    bag_of_words2vec.append(tokens)
bag_of_words2vec

[[0, 1, 1, 0, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1]]

## gensim构建词袋模型

In [9]:
from gensim import corpora
import gensim

In [11]:
dictionary = corpora.Dictionary(tokenized) # 去标点之后的
dictionary.token2id

{'人工智能': 0,
 '发展': 1,
 '和': 8,
 '学习': 2,
 '带动': 3,
 '机器': 4,
 '深度': 7,
 '的': 5,
 '飞速': 6}

In [12]:
corpus = [dictionary.doc2bow(sentence) for sentence in segs_l]
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]

# 词向量

深度学习带给自然语言处理最令人兴奋的突破是词向量（Word Embedding）技术。词向量技术是将词语转化成为稠密向量。

## one-hot encoder的不足

- 词语编码是随机的，向量之间相互独立，看不出词语之间可能存在的关联关系。
- 向量维度的大小取决于语料库中词语的多少，那这个矩阵过于稀疏，并且会造成维度灾难

## word2Vec

In [18]:
from gensim.models import Word2Vec
import jieba

punctuation = [",","。", ":", ";", ".", "'", '"', "’", "?", "/", "-", "+", "&", "(", ")"]
sentences = [
    "长江是中国第一大河，干流全长6397公里（以沱沱河为源），一般称6300公里。流域总面积一百八十余万平方公里，年平均入海水量约九千六百余亿立方米。以干流长度和入海水量论，长江均居世界第三位。",
    "黄河，中国古代也称河，发源于中华人民共和国青海省巴颜喀拉山脉，流经青海、四川、甘肃、宁夏、内蒙古、陕西、山西、河南、山东9个省区，最后于山东省东营垦利县注入渤海。干流河道全长5464千米，仅次于长江，为中国第二长河。黄河还是世界第五长河。",
    "黄河,是中华民族的母亲河。作为中华文明的发祥地,维系炎黄子孙的血脉.是中华民族民族精神与民族情感的象征。",
    "黄河被称为中华文明的母亲河。公元前2000多年华夏族在黄河领域的中原地区形成、繁衍。",
    "在兰州的“黄河第一桥”内蒙古托克托县河口镇以上的黄河河段为黄河上游。",
    "黄河上游根据河道特性的不同，又可分为河源段、峡谷段和冲积平原三部分。 ",
    "黄河,是中华民族的母亲河。"
    ]
sentences = [jieba.lcut(sen) for sen in sentences]
tokenized = []
for sentence in sentences:
    words = []
    for word in sentence:
        if word not in punctuation:
            words.append(word)
    tokenized.append(words)

In [19]:
model = Word2Vec(tokenized, sg=1, size=100, window=5, min_count=2, negative=1, sample=0.001, hs=1, workers=4)

- sg=1 是 skip-gram 算法，对低频词敏感；默认 sg=0 为 CBOW 算法。
- size 是输出词向量的维数，值太小会导致词映射因为冲突而影响结果，值太大则会耗内存并使算法计算变慢，一般值取为100到200之间。
- window 是句子中当前词与目标词之间的最大距离，3表示在目标词前看3-b 个词，后面看 b 个词（b 在0-3之间随机）
- min_count 是对词进行过滤，频率小于 min-count 的单词则会被忽视，默认值为5
- negative 和 sample 可根据训练结果进行微调，sample 表示更高频率的词被随机下采样到所设置的阈值，默认值为 1e-3
- hs=1 表示层级 softmax 将会被使用，默认 hs=0 且 negative 不为0，则负采样将会被选择使用。

In [22]:
model.similarity("黄河", "黄河")

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  if __name__ == '__main__':


1.0

In [24]:
model.similarity("黄河", "长江")

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  if __name__ == '__main__':


-0.13522842980956512

In [25]:
model.most_similar(positive=["黄河", "母亲河"], negative=["长江"])

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('，', 0.17788469791412354),
 ('全长', 0.14078979194164276),
 ('中国', 0.13813526928424835),
 ('水量', 0.05989128351211548),
 ('以', 0.019099431112408638),
 ('干流', 0.00837838277220726),
 ('和', 0.004638686776161194),
 ('长河', -0.008439077995717525),
 ('在', -0.021060697734355927),
 ('的', -0.033146701753139496)]

## Doc2Vec

###  两种实现

- DBOW（Distributed Bag of Words）
- DM （Distributed Memory）

In [27]:
from gensim.models.doc2vec import  Doc2Vec, LabeledSentence

doc_lables = ["长江","黄河","黄河","黄河","黄河","黄河","黄河"]

In [ ]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, lables_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
        
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield LabeledSentence(words=doc,tags=[self.labels_list[idx]])
            
    model = Doc2Vec(documents, dm=1, size=100, window=8, min_count=5, workers=4)
    model.save("model")
    model = Doc2Vec.load("model")

In [ ]:
iter_data = LabeledLineSentence(tokenized, doc_lables)

In [ ]:
model = Doc2Vec(dm=1, size=100, window=8, min_count=5, workers=4)
model.build_vocab(iter_data)

In [ ]:
model.train(iter_data, total_examples=model.corpus_count, epochs=1000, start_alpha=0.01, end_alpha=0.001)

In [ ]:
model.docvecs.most_similar("黄河")

总结，本文只是简单的介绍了词袋和词向量模型的典型应用，对于两者的理论和其他词向量模型，比如 TextRank 、FastText 和 GloVe 等